## Segmenting and Clustering Neighborhoods in Toronto
### Let's get started
import necessary packages


In [183]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
#  http://beautiful-soup-4.readthedocs.io/en/latest/    # for more advanced web scraping  

import lxml
import html5lib

print('Libraries imported.')

Libraries imported.


Get postal code data from Wikipedias page List of "postal codes of Canada: M"  

Then turn it into a panda dataframe

In [184]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, attrs={"class": "wikitable"})[0]   # 0 is for the 1st table in this particular page
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Clean the data in the dataframe df

In [185]:
df['Borough']=df['Borough'].replace('Not assigned',np.NaN)   # replace 'Not assigned' with NaN 
df=df.dropna() 
df.reset_index(drop=True, inplace=True)
print("Dropped 'Not assigned' values in the 'Postal Code' column!")

# pd.set_option('display.max_rows', None)     # show all data in dataframe

# check for mispelled 'Not assigned' values in 'Borough' which has not been replaced
print("Any mispelles not assigned values in 'Borough'?  " + str(df['Borough'][df['Borough'].str.lower().str.contains('t as', regex=False)]) + ", " + str(df['Borough'][df['Borough'].str.lower().str.contains('not ', regex=False)]) )   

# find non unique (duplicate) postal codes 
dupli= "no"
for d in df['Postal Code'].duplicated().unique():
    if d == True:
        dupli = ""
print("There is " + dupli+" duplicate 'Postal Code' rows!")


# find neighborhoods with 'Not assigned' values
assigned="Cannot find any"
for a in df['Neighbourhood'].isin(['Not assigned']):
    if a == True:
        assigned="Found"
print(assigned+" 'Not assigned' values in the 'Neighbourhood' column!")

df.head()

Dropped 'Not assigned' values in the 'Postal Code' column!
Any mispelles not assigned values in 'Borough'?  Series([], Name: Borough, dtype: object), Series([], Name: Borough, dtype: object)
There is no duplicate 'Postal Code' rows!
Cannot find any 'Not assigned' values in the 'Neighbourhood' column!


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Let's check the size of our dataframe

In [186]:
df.shape

(103, 3)

This was evverything for week 3 assignment part 1.

## Part 2

Import necessary packages: 

Google Maps Geocoding API started to cost therefore geocoder is used, however it is a bit unreliable it seems, thus we need to download an excel file with the corresponding data of the coordinates


In [187]:
df_coordinates=pd.read_csv('../datafiles/Geospatial_Coordinates.csv')
df_coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [188]:
a='M9N'
df_coordinates.loc[ df_coordinates['Postal Code'] == a ]['Latitude'].to_list()[0]

43.706876

In [189]:
# pip install geopandas

In [190]:
import geocoder # import geocoder
import geopy
from geopy.geocoders import Nominatim
import sys
import time
from time import sleep
# import geopandas

In [191]:
# https://towardsdatascience.com/geocode-with-python-161ec1e62b89

locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Champ de Mars, Paris, France")

In [192]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode(['{},North York, Toronto, Ontario'.format('M3A')]) # M4A not working

In [193]:
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 43.7543263, Longitude = -79.44911696639593


In [194]:
# g = geocoder.google('{}, Toronto, Ontario'.format("M3A"))
# g

In [195]:
# g = geocoder.mapquest(['{}, Toronto, Ontario'.format('M3A')])

In [196]:
# https://geocoder.readthedocs.io/providers/MapQuest.html#geocoding
# g = geocoder.mapquest(['Mountain View, CA', 'Boulder, Co'], method='batch')

In [197]:
df['Postal Code'].loc[4]

'M7A'

In [198]:
# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

In [200]:
location = locator.geocode('{}, Toronto, Ontario'.format(pcode))
for i, pcode in enumerate(df['Postal Code']):
    print("\n")
    print(i, pcode)

    # initialize your variable to None
    lat_coords =None
    lng_coords =None

    locator = Nominatim(user_agent="myGeocoder")
    count = 0
    # loop until you get the coordinates
    while(lat_coords is None):
        
        
        if(count is 1):
            print("Trying to find "+str(a) + ":")

        if(count is not 0):
                sys.stdout.write('\r')
                sys.stdout.write("[%-10s] %d%%" % ('='*count, 25*count))
                sys.stdout.flush()
                # sleep(0.02)
        if(location is not None):
            print(location)

        try:
            lat_coords = location.latitude
            lng_coords = location.longitude
            print("Coordinates: "+ str(lat_coords) + ", "+str(lng_coords) )
        except:
            lat_coords =None
            lng_coords =None
            # print("Except")
        if count == 4:  # the limit
            print(" Error 'Location' not found in Geocode! Switching to excel file")
            lat_coords = df_coordinates.loc[ df_coordinates['Postal Code'] == pcode ]['Latitude'].to_list()[0]
            lng_coords = df_coordinates.loc[ df_coordinates['Postal Code'] == pcode ]['Longitude'].to_list()[0]
            print("\nCoordinates: "+ str(lat_coords) + ", "+str(lng_coords) )
        count = count + 1

        
    latitude = lat_coords
    longitude = lng_coords



0 M3A
Trying to find M9N:
[====      ] 100% Error 'Location' not found in Geocode! Switching to excel file

Coordinates: 43.7532586, -79.3296565


1 M4A
Trying to find M9N:
[====      ] 100% Error 'Location' not found in Geocode! Switching to excel file

Coordinates: 43.725882299999995, -79.31557159999998


2 M5A
Trying to find M9N:
[====      ] 100% Error 'Location' not found in Geocode! Switching to excel file

Coordinates: 43.6542599, -79.3606359


3 M6A
Trying to find M9N:
[====      ] 100% Error 'Location' not found in Geocode! Switching to excel file

Coordinates: 43.718517999999996, -79.46476329999999


4 M7A
Trying to find M9N:
[====      ] 100% Error 'Location' not found in Geocode! Switching to excel file

Coordinates: 43.6623015, -79.3894938


5 M9A
Trying to find M9N:
[====      ] 100% Error 'Location' not found in Geocode! Switching to excel file

Coordinates: 43.6678556, -79.53224240000002


6 M1B
Trying to find M9N:
[====      ] 100% Error 'Location' not found in Geoco

In [108]:
# LOADING SCREEN
# import sys
# import time
# from time import sleep
# for i in range(101):
#     sys.stdout.write('\r')
#     sys.stdout.write("[%-10s] %d%%" % ('='*i, 1*i))
#     sys.stdout.flush()
#     sleep(0.02)

[====================================================================================================] 100%